In [1]:
import pandas as pd
import numpy as np

In [110]:
# 成人问卷
df1=pd.read_sas('CFPS2016/cfps2016adult_201906.sas7bdat')
df1['pid']=df1['pid'].astype(int)
t1=pd.read_sas('CFPS2016/cfps2016famconf_201804.sas7bdat')
t1['pid']=t1['pid'].astype(int)

In [111]:
def eduy(x):
    if x==1:
        return 0
    elif x==2:
        return 6
    elif x==3:
        return 9
    elif x==4:
        return 12
    elif x==5:
        return 15
    elif x==6:
        return 16
    elif x==7:
        return 19
    elif x==8:
        return 22

In [112]:
out_df1=np.where((df1.pid<100051501)|(df1.pid>2489151502))[0]
df1.drop(out_df1,inplace=True)
df1.reset_index(drop=True,inplace=True)
# 筛选所需变量
t1=t1[['pid','TB4_a16_p']] # 添加了最高学历
# 进行从个人库中匹配子代的相关信息
# 成人库中选取相关变量
'''
pa301：现在的户口状况
school：上学状态
employ：当前工作状态
qn4001：是否党员
QG303CODE_ISEI：职业isei
QP201：健康状况
cfps2016eduy：受教育年限
cfps2016eduy_im：受教育年限插补
incomeb：主要工作收入
incomea：一般工作收入
income：总收入
'''
# df1_1=df1[['pid','provcd16','urban16','CFPS_AGE','CFPS_GENDER','PA301',
#           'SCHOOL','EMPLOY','QN4001','QP201','cfps2016eduy',
#            'cfps2016eduy_im', 'cfps2016edu','INCOMEB','INCOMEA','INCOME','incomeb_imp']]
df1_1=df1[['pid','provcd16','urban16','CFPS_AGE','CFPS_GENDER',
          'EMPLOY','qg303code_isei','cfps2016eduy','cfps2016eduy_im', 
           'cfps2016edu','INCOMEB','INCOMEA','INCOME','incomeb_imp']]

# 获取个人信息
merge1=pd.merge(df1_1,t1,on='pid')
for x in merge1.columns.values:
    merge1.rename(columns={x:x.lower()},inplace=True)
    
# 年龄
index1=np.where((merge1.cfps_age<36)|(merge1.cfps_age>65))[0]
# 工作状态 性别（父代辅助样本）
index2=np.where((merge1.employ==0)|(merge1.cfps_gender==0))[0]
merge1_2=merge1.drop(list(index1)+list(index2))
merge1_2.reset_index(drop=True,inplace=True)

# 由于操作不当，2014-2016两次调查间主要工作未发生变动的人群的主要工作数据未采集成功，通过模型进行了插补
# 如果主要工作收入incomeb=-8缺失，需要看是否有incomeb_imp
# 考虑使用总收入income，因为有些一般工作收入incomea>incomeb，需要全面考虑,income=incomea+incomeb
# 计算前先剔除incomea=-9（缺失）,incomea=-8时取0，income=incomeb
merge1_2['incomeb_adj']=merge1_2.apply(lambda x:x['incomeb_imp'] if x['incomeb']==-8 else x['incomeb'],axis=1)
merge1_2.drop(['incomeb'],axis=1,inplace=True)
merge1_2.rename(columns={'incomeb_adj':'incomeb'},inplace=True)
merge1_2.reset_index(drop=True,inplace=True)
merge1_2[['incomea']]=merge1_2[['incomea']].apply(lambda x:x.replace(-9,np.nan))
merge1_2['income']=merge1_2.apply(lambda x:x['incomea']+x['incomeb'] if ((x['incomea']!=-8)&(pd.notnull(x['incomea']))) else x['incomeb'],axis=1)

# 剔除收入为空，或为-8,0的样本数据，以income为标准，后面要做两样本，所以暂时不管父代收入
merge1_3=merge1_2.dropna(subset=['income']).reset_index(drop=True)
index3=np.where(merge1_3['income']<=0)[0]
merge1_4=merge1_3.drop(index3) # 剩1155
merge1_4.reset_index(drop=True,inplace=True)

# 查看学历是否需要插补
# 若教育年限缺失，看是否能用插补值；若最高学历缺失，考虑用最近一次调查的最高学历
# 注意插补后有为-8和空的，后续记得剔除
merge1_5=merge1_4.copy()
merge1_5['cfps2016eduy']=merge1_5.apply(lambda x:x['cfps2016eduy_im'] if np.isnan(x['cfps2016eduy']) else x['cfps2016eduy'],axis=1)
merge1_5['cfps2016edu']=merge1_5.apply(lambda x:x['tb4_a16_p'] if np.isnan(x['cfps2016edu']) else x['cfps2016edu'],axis=1)
merge1_5['cfps2016eduy']=merge1_5.apply(lambda x:eduy(x['cfps2016edu']) if np.isnan(x['cfps2016eduy']) else x['cfps2016eduy'],axis=1)
merge1_5.drop(['cfps2016eduy_im'],axis=1,inplace=True)

# 根据其他相关变量剔除样本，包括缺失、不适用等
merge1_6=merge1_5.dropna(subset=['provcd16','urban16','cfps_gender','cfps2016eduy','cfps2016edu','qg303code_isei']).reset_index(drop=True)
index4=index4=np.where((merge1_6.provcd16==-9)|(merge1_6.cfps_gender==-8)|(merge1_6.urban16==-9))[0]
merge1_6.drop(index4,inplace=True)
merge1_6.reset_index(drop=True,inplace=True)# 剩1121
merge1_6['year']=2016

merge1_6.drop(['pid','employ','incomea','incomeb','incomeb_imp','tb4_a16_p','cfps_gender'],axis=1,inplace=True)
merge1_6.rename(columns={'provcd16':'pro','urban16':'urban','cfps_age':'age','qg303code_isei':'job','cfps2016eduy':'eduy','cfps2016edu':'edu'},inplace=True)

In [114]:
# 成人问卷
df2=pd.read_sas('CFPS2014/cfps2014adult_201906.sas7bdat')
df2['pid']=df2['pid'].astype(int)
t2=pd.read_sas('CFPS2014/cfps2014famconf_170630.sas7bdat')
t2['pid']=t2['pid'].astype(int)
# 2016年跨年库
c2=pd.read_sas('CFPS2016/cfps2016crossyearid_201807.sas7bdat')
c2['pid']=c3['pid'].astype(int)

In [115]:
out_df2=np.where((df2.pid<100051501)|(df2.pid>2489151502))[0]
df2.drop(out_df2,inplace=True)
df2.reset_index(drop=True,inplace=True)
t2=t2[['pid','TB4_a14_p']]
# 成人库中选取相关变量
'''
qa301：现在的户口状况
wc01：上学状态
employ2014：当前工作状态
qn401_s_1：您是哪些组织成员，党员为1，其余的后面设为0，剔除缺失
QP201：健康状况
cfps2014edu：最高学历
cfps2014eduy：受教育年限
cfps2014eduy_im：受教育年限插补
income：总收入
'''
df2=pd.merge(df2,c2[['pid','INCOME_14']],on='pid',how='left')
df2_1=df2[['pid','provcd14','urban14','CFPS2014_AGE','CFPS_GENDER',
              'EMPLOY2014','QG303CODE','cfps2014eduy',
           'cfps2014eduy_im', 'cfps2014edu','INCOME','INCOME_14']]
merge2=pd.merge(df2_1,t2,on='pid')
for x in merge2.columns.values:
    merge2.rename(columns={x:x.lower()},inplace=True)
# 年龄
index1=np.where((merge2.cfps2014_age<36)|(merge2.cfps2014_age>65))[0]
# 工作状态 性别（父代辅助样本）
index2=np.where((merge2.employ2014==0)|(merge2.cfps_gender==0))[0]
merge2_2=merge2.drop(list(index1)+list(index2))
merge2_2.reset_index(drop=True,inplace=True)
# 用2016跨年库中的income_14对收入进行插补
merge2_2['income']=merge2_2.apply(lambda x:x['income_14'] if ((np.isnan(x['income']))|(x['income']==-8)|(x['income']==0))
                            else x['income'],axis=1)
# 剔除收入为空，或小于等于0的样本数据，以income为标准
merge2_3=merge2_2.dropna(subset=['income']).reset_index(drop=True)
index4=np.where((merge2_3['income']<=0))[0]
merge2_4=merge2_3.drop(index4) # 剩1269
merge2_4.reset_index(drop=True,inplace=True)
# 查看学历是否需要插补
# 先把所有-9（缺失）替换为NA
# 插补后记得剔除edu为0的，表示没有数据
merge2_5=merge2_4.copy()
merge2_5[['cfps2014edu','cfps2014eduy','cfps2014eduy_im']]=merge2_5[['cfps2014edu','cfps2014eduy','cfps2014eduy_im']].apply(lambda x:x.replace(-9,np.nan))
merge2_5['cfps2014eduy']=merge2_5.apply(lambda x:x['cfps2014eduy_im'] if ((np.isnan(x['cfps2014eduy']))|(x['cfps2014eduy']==-1)) else x['cfps2014eduy'],axis=1)
merge2_5['cfps2014edu']=merge2_5.apply(lambda x:x['tb4_a14_p'] if ((np.isnan(x['cfps2014edu']))|(x['cfps2014edu']==-1)) else x['cfps2014edu'],axis=1)
merge2_5['cfps2014eduy']=merge2_5.apply(lambda x:eduy(x['cfps2014edu']) if ((np.isnan(x['cfps2014eduy']))|(x['cfps2014eduy']==-1)) else x['cfps2014eduy'],axis=1)
merge2_5.drop(['cfps2014eduy_im'],axis=1,inplace=True)
# 根据其他相关变量剔除样本，包括缺失、不适用等
merge2_6=merge2_5.dropna(subset=['provcd14','urban14','qg303code','cfps2014edu','cfps2014eduy','cfps_gender']).reset_index(drop=True)
index5=np.where((merge2_6.provcd14==-9)|(merge2_6.qg303code.isin([-1,-2,-8,-9]))|
            (merge2_6.urban14==-9)|(merge2_6.cfps_gender==-8)|
            (merge2_6.cfps2014edu==-1))[0]
merge2_6.drop(index5,inplace=True)
merge2_6.reset_index(drop=True,inplace=True)# 剩1070
merge2_6['year']=2014
merge2_6.drop(['pid','cfps_gender','employ2014','income_14','tb4_a14_p'],axis=1,inplace=True)
merge2_6.rename(columns={'provcd14':'pro','cfps2014_age':'age','urban14':'urban','qg303code':'job','cfps2014eduy':'eduy',
                        'cfps2014edu':'edu'},inplace=True)

In [118]:
# # 成人问卷
df3=pd.read_sas('CFPS2012/cfps2012adult_201906.sas7bdat')
df3['pid']=df3['pid'].astype(int)
t3=pd.read_sas('CFPS2012/cfps2012famconf_092015.sas7bdat')
t3['pid']=t3['pid'].astype(int)
# 2016跨年数据
c3=pd.read_sas('CFPS2016/cfps2016crossyearid_201807.sas7bdat')
c3['pid']=c3['pid'].astype(int)

In [119]:
out_df3=np.where((df3.pid<105179433)|(df3.pid>997150431))[0]
df3.drop(out_df3,inplace=True)
df3.reset_index(drop=True,inplace=True)
t3=t3[['pid','TB4_A12_p']]

# 获取个人信息
# 先匹配跨年库中的相关变量
df3=pd.merge(df3,c3[['pid','INCOME_12','INCOME_ADJ_12','cfps2012edu',
'cfps2012eduy','cfps2012eduy_im']],on='pid',how='left')
'''
qa301：现在的户口状况
wc01：上学状态
employ：当前工作状态
QN401_S_1：您是哪些组织成员，党员为1，其余的后面设为0，剔除缺失
QP201：健康状况
edu2012：最高学历
eduy2012:受教育年限
cfps2012edu:最高学历
cfps2012eduy：受教育年限(2012跨年库中)
cfps2012eduy_im：受教育年限插补
income：总收入
job2012mn_occu：当前主要职业代码国标
'''
df3_1=df3[['pid','provcd','urban12','CFPS2012_AGE','CFPS2012_GENDER','EMPLOY',
           'job2012mn_occu','edu2012','eduy2012','INCOME','income_adj',
          'INCOME_12','INCOME_ADJ_12','cfps2012edu','cfps2012eduy','cfps2012eduy_im']]
merge3=pd.merge(df3_1,t3,on='pid')
for x in merge3.columns.values:
    merge3.rename(columns={x:x.lower()},inplace=True)
# 年龄
index1=np.where((merge3.cfps2012_age<36)|(merge3.cfps2012_age>65))[0]
# 工作状态 性别（父代辅助样本）
index2=np.where((merge3.employ==0)|(merge3.cfps2012_gender==0))[0]
merge3_2=merge3.drop(list(index1)+list(index2))
merge3_2.reset_index(drop=True,inplace=True)
# 用2016跨年库中的income_adj_12对收入进行插补
merge3_2['income_adj']=merge3_2.apply(lambda x:x['income_adj_12'] if ((np.isnan(x['income_adj']))|(x['income_adj']==-8)|(x['income_adj']==0))
                            else x['income_adj'],axis=1)
# 剔除收入为空，或为-8,0的样本数据，以income_adj为标准
# 由于income_adj数据质量更好，使用该变量，删除income，并将income_adj进行更名
merge3_2.drop(['income'],axis=1,inplace=True)
merge3_2.rename(columns={'income_adj':'income'},inplace=True)
merge3_3=merge3_2.dropna(subset=['income']).reset_index(drop=True)
index3=np.where((merge3_3['income']<=0))[0]
merge3_4=merge3_3.drop(index3) # 剩1617
merge3_4.reset_index(drop=True,inplace=True)
# 查看学历是否需要插补
# 先把所有-9（缺失）替换为NA
# 插补后记得剔除edu为0的，表示没有数据
merge3_5=merge3_4.copy()
merge3_5['eduy2012']=merge3_5.apply(lambda x:x['cfps2012eduy_im'] if x['cfps2012eduy_im']>x['eduy2012'] else x['eduy2012'],axis=1)
merge3_5['edu2012']=merge3_5.apply(lambda x:x['cfps2012edu'] if x['cfps2012edu']>x['edu2012'] else x['edu2012'],axis=1)
merge3_5['edu2012']=merge3_5.apply(lambda x:x['tb4_a12_p'] if ((np.isnan(x['edu2012']))|(x['edu2012']==-1)) else x['edu2012'],axis=1)
merge3_5['eduy2012']=merge3_5.apply(lambda x:eduy(x['edu2012']) if ((np.isnan(x['eduy2012']))|(x['eduy2012']==-1)) else x['eduy2012'],axis=1)
merge3_5.drop(['cfps2012eduy_im','cfps2012edu','cfps2012eduy'],axis=1,inplace=True)
# 根据其他相关变量剔除样本，包括缺失、不适用等
merge3_6=merge3_5.dropna(subset=['provcd','urban12','job2012mn_occu',
                         'edu2012','eduy2012','cfps2012_gender']).reset_index(drop=True)
index4=np.where((merge3_6.provcd==-9)|(merge3_6.job2012mn_occu==-8)|
                (merge3_6.urban12==-9))[0]
merge3_6.drop(index4,inplace=True)
merge3_6.reset_index(drop=True,inplace=True)# 剩1070
merge3_6['year']=2012
merge3_6.drop(['pid','cfps2012_gender','employ','income_12','income_adj_12','tb4_a12_p'],axis=1,inplace=True)
merge3_6.rename(columns={'provcd':'pro','cfps2012_age':'age','urban12':'urban','job2012mn_occu':'job','eduy2012':'eduy',
                        'edu2012':'edu'},inplace=True)

In [121]:
# # 成人问卷
df4=pd.read_sas('CFPS2010/cfps2010adult_201906.sas7bdat')
df4['pid']=df4['pid'].astype(int)
t4=pd.read_sas('CFPS2010/cfps2010famconf_report_nat072016.sas7bdat')
t4['pid']=t4['pid'].astype(int)
# 2016年跨年库
c4=pd.read_sas('CFPS2016/cfps2016crossyearid_201807.sas7bdat')
c4['pid']=c4['pid'].astype(int)

In [122]:
out_df4=np.where((df4.pid<110001101)|(df4.pid>621872103))[0]
df4.drop(out_df4,inplace=True)
df4.reset_index(drop=True,inplace=True)
t4=t4[['pid','TB4_A_p']]
# 获取个人信息
# 先匹配跨年库中的相关变量
df4=pd.merge(df4,c4[['pid','INCOME_10','cfps2010eduy_im','cfps2010eduy']],on='pid',how='left')
'''
QA2：现在的户口状况
QD3：上学状态
QG3：当前工作状态
QA7_S_1：您是哪些组织成员，党员为1，其余的后面设为0，剔除缺失
QP3：健康状况
cfps2010edu_best:最高学历
cfps2010eduy_best：受教育年限(2012跨年库中)
cfps2010eduy_im：受教育年限插补
income：总收入
'''
df4_1=df4[['pid','provcd','urban','QA1AGE','GENDER','QG3','qg307isei','INCOME','INCOME_10',
           'cfps2010edu_best','cfps2010eduy_best','cfps2010eduy_im']]
merge4=pd.merge(df4_1,t4,on='pid')
for x in merge4.columns.values:
    merge4.rename(columns={x:x.lower()},inplace=True)

# 年龄
index1=np.where((merge4.qa1age<36)|(merge4.qa1age>65))[0]
# 工作状态 性别（父代辅助样本）
index2=np.where((merge4.qg3==0)|(merge4.gender==0))[0]
merge4_2=merge4.drop(list(index1)+list(index2))
merge4_2.reset_index(drop=True,inplace=True)

# 用2016跨年库中的income_10对收入进行插补
merge4_2['income']=merge4_2.apply(lambda x:x['income_10'] if ((np.isnan(x['income']))|(x['income']==-8)|(x['income']==0))
                            else x['income'],axis=1)
# 剔除收入为空，或为-8,0的样本数据，以income为标准
merge4_3=merge4_2.dropna(subset=['income']).reset_index(drop=True)
index3=np.where((merge4_3['income']<=0))[0]
merge4_4=merge4_3.drop(index3) # 剩1522
merge4_4.reset_index(drop=True,inplace=True)
# 查看学历是否需要插补
# 先把所有-9（缺失）替换为NA
# 插补后记得剔除edu为0的，表示没有数据
merge4_5=merge4_4.copy()
merge4_5['cfps2010eduy_best']=merge4_5.apply(lambda x:x['cfps2010eduy_im'] if ((np.isnan(x['cfps2010eduy_best']))|(x['cfps2010eduy_best']==-1)) else x['cfps2010eduy_best'],axis=1)
merge4_5['cfps2010edu_best']=merge4_5.apply(lambda x:x['tb4_a_p'] if ((np.isnan(x['cfps2010edu_best']))|(x['cfps2010edu_best']==-1)) else x['cfps2010edu_best'],axis=1)
merge4_5['cfps2010eduy_best']=merge4_5.apply(lambda x:eduy(x['cfps2010edu_best']) if ((np.isnan(x['cfps2010eduy_best']))|(x['cfps2010eduy_best']==-1)) else x['cfps2010eduy_best'],axis=1)
merge4_5.drop(['cfps2010eduy_im'],axis=1,inplace=True)
# 根据其他相关变量剔除样本，包括缺失、不适用等
merge4_6=merge4_5.dropna(subset=['provcd','urban','qg307isei',
                         'cfps2010edu_best','cfps2010eduy_best']).reset_index(drop=True)
index4=np.where((merge4_6.provcd==-9)|(merge4_6.urban==-9)|(merge4_6.qg307isei<0)|
            (merge4_6.cfps2010edu_best==-1)|(merge4_6.cfps2010eduy_best==-1))[0]
merge4_6.drop(index4,inplace=True)
merge4_6.reset_index(drop=True,inplace=True)# 剩1519
merge4_6['year']=2010
merge4_6.drop(['pid','gender','qg3','income_10','tb4_a_p'],axis=1,inplace=True)
merge4_6.rename(columns={'provcd':'pro','qa1age':'age','qg307isei':'job','cfps2010edu_best':'edu',
                        'cfps2010eduy_best':'eduy'},inplace=True)

In [105]:
cpi=pd.read_csv('data result/cpi_2009.csv',encoding='GBK',index_col=0).reset_index()

In [133]:
res1=pd.merge(merge1_6,cpi,left_on='pro',right_on='No')
res2=pd.merge(merge2_6,cpi,left_on='pro',right_on='No')
res3=pd.merge(merge3_6,cpi,left_on='pro',right_on='No')
res4=pd.merge(merge4_6,cpi,left_on='pro',right_on='No')

In [134]:
# 调整价格，2009为基期
res1['income']=res1.apply(lambda x:x['income']/x['2016'],axis=1)
res2['income']=res2.apply(lambda x:x['income']/x['2014'],axis=1)
res3['income']=res3.apply(lambda x:x['income']/x['2012'],axis=1)
res4['income']=res4.apply(lambda x:x['income']/x['2010'],axis=1)

In [135]:
for x in [res1,res2,res3,res4]:
    x.drop(['2009','2010','2011','2012','2013','2014','2015','2016','2017'],axis=1,inplace=True)

In [137]:
res1.to_csv('data result/model2016.csv',index=False)
res2.to_csv('data result/model2014.csv',index=False)
res3.to_csv('data result/model2012.csv',index=False)
res4.to_csv('data result/model2010.csv',index=False)